In [ ]:
from finance.data.db.mysql import MySQLClient
from finance.data.fundamentals import upsert_fundamentals
from finance.data.factors import  upsert_factors
from finance.data.asset_profile import load_symbols_from_asset_profile
from edgar import Company, set_identity

import pandas as pd


[
    'Total Revenue',                                # 총매출 (회사 전체의 매출액)
    'Cost of Revenue',                              # 매출원가 (제품/서비스 판매에 소요된 비용)
    'Gross Profit',                                 # 매출총이익 (총매출 - 매출원가)
    'Operating expenses:',                          # 영업비용 (운영에 소요된 비용)
    'Research and Development Expense',             # 연구개발비 (신제품/서비스 개발비용)
    'Selling and Marketing Expense',                # 판매 및 마케팅 비용
    'General and Administrative Expense',           # 일반관리비 (경영, 행정 비용)
    'Operating Expenses',                           # 영업비용 전체
    'Selling, General and Administrative Expense',  # 판매 관리비 (SG&A)
    'Operating Income (Loss)',                      # 영업이익 (손실)
    'Other Nonoperating Income (Expense)',          # 기타 영업외 수익(비용)
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest',  # 계속영업이익 (법인세 차감 전, 비지배 지분 전)
    'Income Tax Expense (Benefit)',                 # 법인세 비용(혜택)
    'Net Income (Loss) Attributable to Parent',     # 순이익 (지배기업 귀속)
    'Earnings Per Share, Basic',                    # 주당 순이익 (기본)
    'Earnings Per Share, Diluted',                  # 주당 순이익 (희석)
    'Other income (expense):',                      # 기타수익(비용)
    'Nonoperating Income (Expense)',                # 영업외 수익(비용)
    'Additional Financial Items',                   # 추가 재무 항목
    'Amortization of Intangible Assets',            # 무형자산상각비
    'Common Stock, Dividends, Per Share, Declared', # 보통주 1주당 배당금
    'Depreciation, Depletion and Amortization, Nonproduction',  # 감가상각비(비생산활동 관련)
    'Goodwill, Impairment Loss',                    # 영업권 손상차손
    'Interest Expense',                             # 이자 비용
    'Interest Expense, Debt',                       # 부채 이자 비용
    'Reclassification from AOCI, Current Period, before Tax, Attributable to Parent', # 기타포괄손익누계액의 재분류 (지배기업 귀속)
    'Revenue, Net (Deprecated 2018-01-31)'          # 순매출(구버전, 더 이상 사용안함)
]

financial_term_kor_map = {
    'Total Revenue': '총매출 (회사 전체의 매출액)',
    'Cost of Revenue': '매출원가 (제품/서비스 판매에 소요된 비용)',
    'Gross Profit': '매출총이익 (총매출 - 매출원가)',
    'Operating expenses:': '영업비용 (운영에 소요된 비용)',
    'Research and Development Expense': '연구개발비 (신제품/서비스 개발비용)',
    'Selling and Marketing Expense': '판매 및 마케팅 비용',
    'General and Administrative Expense': '일반관리비 (경영, 행정 비용)',
    'Operating Expenses': '영업비용 전체',
    'Selling, General and Administrative Expense': '판매 관리비 (SG&A)',
    'Operating Income (Loss)': '영업이익 (손실)',
    'Other Nonoperating Income (Expense)': '기타 영업외 수익(비용)',
    'Income (Loss) from Continuing Operations before Income Taxes, Noncontrolling Interest': '계속영업이익 (법인세 차감 전, 비지배 지분 전)',
    'Income Tax Expense (Benefit)': '법인세 비용(혜택)',
    'Net Income (Loss) Attributable to Parent': '순이익 (지배기업 귀속)',
    'Earnings Per Share, Basic': '주당 순이익 (기본)',
    'Earnings Per Share, Diluted': '주당 순이익 (희석)',
    'Other income (expense):': '기타수익(비용)',
    'Nonoperating Income (Expense)': '영업외 수익(비용)',
    'Additional Financial Items': '추가 재무 항목',
    'Amortization of Intangible Assets': '무형자산상각비',
    'Common Stock, Dividends, Per Share, Declared': '보통주 1주당 배당금',
    'Depreciation, Depletion and Amortization, Nonproduction': '감가상각비(비생산활동 관련)',
    'Goodwill, Impairment Loss': '영업권 손상차손',
    'Interest Expense': '이자 비용',
    'Interest Expense, Debt': '부채 이자 비용',
    'Reclassification from AOCI, Current Period, before Tax, Attributable to Parent': '기타포괄손익누계액의 재분류 (지배기업 귀속)',
    'Revenue, Net (Deprecated 2018-01-31)': '순매출(구버전, 더 이상 사용안함)',
    'Business Combination, Acquisition Related Costs': '사업결합, 인수 관련 비용',
    'Cost of Goods and Service, Excluding Depreciation, Depletion, and Amortization': '감가상각/고갈/무형자산상각 제외 상품 및 서비스 원가',
    'Cost of Goods Sold (Deprecated 2018-01-31)': '매출원가(구버전, 더 이상 사용안함)',
    'Current Income Tax Expense (Benefit)': '당기 법인세 비용(혜택)',
    'Equity Securities without Readily Determinable Fair Value, Impairment Loss, Annual Amount': '공정가치 산정 불가 지분증권 손상차손 (연간)',
    'Income (Loss) from Continuing Operations, Net of Tax, Attributable to Parent': '계속영업이익(세후, 지배기업 귀속)',
    'Income (Loss) from Continuing Operations before Income Taxes, Domestic': '국내 계속영업이익 (법인세 이전)',
    'Income (Loss) from Continuing Operations, Per Basic Share': '계속영업 주당 순이익(기본)',
    'Income (Loss) from Continuing Operations, Per Diluted Share': '계속영업 주당 순이익(희석)',
    'Income (Loss) from Discontinued Operations, Net of Tax, Attributable to Parent': '중단영업이익(손실, 세후, 지배기업 귀속)',
    'Income (Loss) from Equity Method Investments': '지분법 투자이익(손실)',
    'Interest Income (Expense), Net': '이자수익(비용) 순액',
    'Marketing and Advertising Expense': '마케팅 및 광고 비용',
    'Operating Lease, Expense': '운영리스 비용',
    'Restructuring and Related Cost, Incurred Cost': '구조조정 및 관련 비용, 확정된 비용',
    'Restructuring Costs': '구조조정 비용',
    'Sales Revenue, Goods, Net (Deprecated 2018-01-31)': '상품 순매출 (구버전, 더 이상 사용안함)',
    'Net Income (Loss) Attributable to Noncontrolling Interest': '순이익(비지배지분 귀속)',
    'Investment Income, Interest': '이자성 투자수익',
    'Foreign Currency Transaction Gain (Loss), before Tax': '외화거래이익(손실, 세전)',
    'Foreign Currency Transaction Gain (Loss), Realized': '외화거래이익(손실, 실현)',
    'Gain (Loss) on Investments': '투자이익(손실)',
    'Net Income (Loss) Available to Common Stockholders, Basic': '보통주주 귀속 순이익(손실, 기본)',
    'Expenses': '비용',
    'Policyholder Benefits and Claims Incurred, Net': '보험계약자 지급 및 발생손해, 순액',
    'Deferred Policy Acquisition Costs, Amortization Expense': '이연 보험계약 인수비용 상각',
    'Benefits, Losses and Expenses': '보험급여, 손실 및 비용',
    'Other Cost and Expense, Operating': '기타 운영비용',
    'Capitalized Computer Software, Amortization': '자본화된 소프트웨어 상각',
    'Equity Securities, FV-NI, Realized Gain (Loss)': '공정가치-당기손익 지분증권, 실현손익',
    'Investment Income, Net': '순투자수익',
    'Liability for Unpaid Claims and Claims Adjustment Expense, Incurred Claims': '미지급 보험금 및 청구조정비용 발생손해',
    'Other Expenses': '기타 비용',
    'Preferred Stock Dividends, Income Statement Impact': '우선주 배당금, 손익계산서 영향',
    'Accounts Payable, Current': '유동 매입채무',
    'Accounts Receivable': '매출채권',
    'Accounts Receivable, after Allowance for Credit Loss': '대손충당금 차감 후 매출채권',
    'Accounts Receivable, after Allowance for Credit Loss, Current': '유동 대손충당금 차감 후 매출채권',
    'Accretion Expense': '적립비용',
    'Accrued Liabilities, Current': '유동 미지급 부채',
    'Accrued capital expenditures': '미지급 자본적 지출',
    'Accumulated Other Comprehensive (Income) Loss, Defined Benefit Plan, after Tax': '기타포괄손익누계액(확정급여형 퇴직급여제도/세후)',
    'Accumulated Other Comprehensive Income (Loss), Net of Tax': '누적 기타포괄손익(순, 세후)',
    'Additional Paid in Capital': '자본잉여금',
    'Additional Paid in Capital, Common Stock': '자본잉여금(보통주)',
    'Amortization': '감가상각',
    'Amortization of Debt Issuance Costs': '부채발행비용 상각',
    'Amortization of Debt Issuance Costs and Discounts': '부채발행비용 및 할인의 상각',
    'Asset Retirement Obligation, Accretion Expense': '자산철거의무 적립비용',
    'Assets': '자산',
    'Assets, Current': '유동자산',
    'Bank Owned Life Insurance Income': '은행 소유 생명보험 수익',
    'Cash and Cash Equivalents, at Carrying Value': '현금및현금성자산(장부가액)',
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents': '현금·현금성자산·제한적 현금 및 현금성자산',
    'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect': '기준기간 중 현금 등 증감(환율효과 포함)',
    'Cash, Cash Equivalents, and Short-term Investments': '현금∙현금성자산 및 단기투자자산',
    'Common Stock, Value, Issued': '발행 보통주 자본금',
    'Cost of Goods and Services Sold': '매출원가',
    'Cost of Property Repairs and Maintenance': '자산 수리 및 유지보수비',
    'Cost, Depreciation and Amortization': '원가, 감가상각 및 상각비',
    'Costs and Expenses': '비용 및 경비',
    'Debt Securities, Available-for-sale, Realized Gain (Loss)': '매도가능증권 실현이익(손실)',
    'Debt Securities, Realized Gain (Loss)': '채무증권 실현이익(손실)',
    'Debt and Equity Securities, Gain (Loss)': '채권 및 주식증권 이익(손실)',
    'Deferred Compensation Equity': '이연보상자본',
    'Deferred Income Tax Expense (Benefit)': '이연법인세 비용(혜택)',
    'Deferred Income Tax Liabilities, Net': '순이연법인세부채',
    'Deferred Revenue, Current': '유동이연수익',
    'Deferred Revenue, Noncurrent': '비유동이연수익',
    'Defined Benefit Plan, Net Periodic Benefit Cost (Credit), Gain (Loss) Due to Settlement': '확정급여형 퇴직연금정산 순주기적급여비용(수익), 정산에 의한 이익(손실)',
    'Defined Benefit Plan, Net Periodic Benefit Cost (Credit), Gain (Loss) Due to Settlement and Curtailment': '확정급여형 퇴직연금정산·감축에 의한 이익(손실)',
    'Depreciation': '감가상각',
    'Depreciation and amortization': '감가상각 및 상각',
    'Depreciation, Amortization and Accretion, Net': '감가상각 및 적립(순)',
    'Depreciation, Depletion and Amortization': '감가상각, 고갈 및 상각',
    'Depreciation, Nonproduction': '비생산 감가상각',
    'Derivative, Gain (Loss) on Derivative, Net': '파생상품 순이익(손실)',
    'Discontinued Operation, Gain (Loss) on Disposal of Discontinued Operation, Net of Tax': '중단사업 처분이익(손실)(세후)',
    'Discontinued Operation, Income (Loss) from Discontinued Operation During Phase-out Period, Net of Tax': '중단사업 폐지 기간 중 손익(세후)',
    'Disposal Group, Not Discontinued Operation, Gain (Loss) on Disposal': '비중단 매각집단 처분이익(손실)',
    'Dividends Payable, Current': '유동미지급배당금',
    'Dividends paid': '지급배당금',
    'Effect of Exchange Rate on Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents': '환율변동이 현금등에 미치는 영향',
    'Environmental Remediation Expense': '환경정화비용',
    'Equity Method Investment, Other than Temporary Impairment': '지분법적용투자 일시적이외의 손상',
    'Equity Securities, FV-NI, Gain (Loss)': '공정가치-순이익지정주식 이익(손실)',
    'Equity Securities, FV-NI, Unrealized Gain (Loss)': '공정가치-순이익지정주식 미실현이익(손실)',
    'Fair Value, Option, Changes in Fair Value, Gain (Loss)': '공정가치 옵션 평가손익',
    'Financing Receivable, Credit Loss, Expense (Reversal)': '금융채권 신용손실비용(환입)',
    'Foreign Currency Transaction Gain (Loss), Unrealized': '외화거래 미실현이익(손실)',
    'Fuel Costs': '연료비',
    'Gain (Loss) Related to Litigation Settlement': '소송합의 관련 이익(손실)',
    'Gain (Loss) on Derivative Instruments, Net, Pretax': '파생상품 순이익(손실)(법인세차감전)',
    'Gain (Loss) on Disposition of Assets': '자산처분이익(손실)',
    'Gain (Loss) on Disposition of Business': '사업처분이익(손실)',
    'Gain (Loss) on Disposition of Other Assets': '기타자산 처분이익(손실)',
    'Gain (Loss) on Disposition of Property Plant Equipment': '유형자산 처분이익(손실)',
    'Gain (Loss) on Disposition of Property Plant Equipment, Excluding Oil and Gas Property and Timber Property': '유형자산 처분이익(손실, 유전·임야제외)',
    'Gain (Loss) on Extinguishment of Debt': '부채소멸이익(손실)',
    'Gain (Loss) on Sale of Assets and Asset Impairment Charges': '자산매각이익(손실) 및 자산손상차손',
    'Gain (Loss) on Sale of Derivatives': '파생상품 매각이익(손실)',
    'Gain (Loss) on Sale of Investments': '투자자산 매각이익(손실)',
    'Gain (Loss) on Sale of Mortgage Loans': '모기지대출 매각이익(손실)',
    'Gain (Loss) on Sale of Properties': '재산 매각이익(손실)',
    'Gain (Loss) on Sales of Loans, Net': '대출금 매각순이익(손실)',
    'Gain (Loss) on Termination of Lease': '리스종료 이익(손실)',
    'Gains (Losses) on Sales of Investment Real Estate': '투자부동산 매각이익(손실)',
    'Goodwill': '영업권',
    'Goodwill and Intangible Asset Impairment': '영업권 및 무형자산 손상',
    'Gross Profit (Calculated)': '매출총이익(계산)',
    'Impairment of Intangible Assets (Excluding Goodwill)': '무형자산손상(영업권제외)',
    'Impairment of Intangible Assets, Finite-lived': '내용연수유한 무형자산 손상',
    'Impairment of Intangible Assets, Indefinite-lived (Excluding Goodwill)': '내용연수무한 무형자산손상(영업권제외)',
    'Income (Loss) Attributable to Parent, before Tax': '모회사 귀속세전이익(손실)',
    'Income (Loss) from Continuing Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest': '지속영업 이익(손실)(비지배지분포함, 세후)',
    'Income (Loss) from Discontinued Operations, Net of Tax, Including Portion Attributable to Noncontrolling Interest': '중단사업 이익(손실)(비지배지분포함, 세후)',
    'Income (Loss) from Equity Method Investments, Net of Dividends or Distributions': '지분법투자이익(손실)(배당차감후)',
    'Income Taxes Paid, Net': '법인세납부(순)',
    'Increase (Decrease) in Accounts Payable': '매입채무증가(감소)',
    'Increase (Decrease) in Accounts Receivable': '매출채권증가(감소)',
    'Increase (Decrease) in Inventories': '재고자산증가(감소)',
    'Intangible Assets, Net (Excluding Goodwill)': '무형자산 순액(영업권제외)',
    'Interest Income (Expense), Nonoperating, Net': '영업외이자수익(비용,순)',
    'Interest Income, Operating': '영업이자수익',
    'Interest Income, Other': '기타이자수익',
    'Interest Paid, Excluding Capitalized Interest, Operating Activities': '영업현금흐름 내 이자지급액(자본화이자제외)',
    'Interest and Other Income': '이자 및 기타수익',
    'InterestExpenseNonoperating': '영업외이자비용',
    'InterestExpenseOperating': '영업이자비용',
    'Inventory Write-down': '재고자산감액',
    'Inventory, Net': '순재고자산',
    'Labor and Related Expense': '인건비 및 관련비용',
    'Liabilities': '부채',
    'Liabilities and Equity': '부채 및 자기자본',
    'Liabilities, Current': '유동부채',
    'Liabilities, Noncurrent': '비유동부채',
    'Long Term Debt': '장기부채',
    'Long-term Debt': '장기부채',
    'Long-term Debt, Current Maturities': '유동성장기부채',
    'Long-term Debt, Excluding Current Maturities': '비유동장기부채',
    'Management Fee Expense': '운용수수료비',
    'Marketable Securities, Realized Gain (Loss), Excluding Other-than-temporary Impairment Loss': '시장성유가증권실현이익(손실)(일시적손상제외)',
    'Marketable Securities, Unrealized Gain (Loss)': '시장성유가증권미실현이익(손실)',
    'Net Cash Provided by (Used in) Financing Activities': '재무활동 현금흐름',
    'Net Cash Provided by (Used in) Investing Activities': '투자활동 현금흐름',
    'Net Cash Provided by (Used in) Operating Activities': '영업활동 현금흐름',
    'Net Income (Loss) from Continuing Operations Available to Common Shareholders, Basic': '지속영업 보통주주귀속순이익(기본)',
    'Net Income (Loss), Including Portion Attributable to Noncontrolling Interest': '순이익(손실)(비지배지분 포함)',
    'Net Investment Income': '순투자이익',
    'Net Periodic Defined Benefits Expense (Reversal of Expense), Excluding Service Cost Component': '확정급여 정기순비용(환입, 서비스비용제외)',
    'Noncash Merger Related Costs': '비현금합병관련비용',
    'Noncontrolling Interest in Net Income (Loss) Operating Partnerships, Redeemable': '상환가능운영파트너십 비지배지분순이익(손실)',
    'Operating Costs and Expenses': '영업비용 및 경비',
    'Operating Lease, Impairment Loss': '운용리스 손상차손',
    'Operating Lease, Lease Income': '운용리스 수익',
    'Operating Lease, Liability, Current': '운용리스 유동부채',
    'Operating Lease, Liability, Noncurrent': '운용리스 비유동부채',
    'Operating Lease, Right-of-Use Asset': '운용리스사용권자산',
    'Other Assets, Noncurrent': '기타비유동자산',
    'Other Cost of Operating Revenue': '기타영업비용',
    'Other Depreciation and Amortization': '기타감가상각',
    'Other General and Administrative Expense': '기타일반관리비',
    'Other Income': '기타수익',
    'Other Liabilities, Noncurrent': '기타비유동부채',
    'Other Operating Income (Expense), Net': '기타영업수익(비용,순)',
    'Parent Company': '모회사',
    "Partners' Capital": '파트너스자본',
    'Payment, Tax Withholding, Share-based Payment Arrangement': '세금원천징수, 주식보상지급',
    'Payments for (Proceeds from) Other Investing Activities': '기타투자활동으로 인한 현금유출(유입)',
    'Payments for Repurchase of Common Stock': '보통주 자사주매입지급',
    'Payments of Dividends': '배당금지급',
    'Payments of Financing Costs': '재무비용지급',
    'Payments of Ordinary Dividends, Common Stock': '보통주 일반배당금 지급',
    'Payments to Acquire Businesses, Net of Cash Acquired': '인수대금 순유출(취득현금차감후)',
    'Payments to Acquire Investments': '투자자산취득지급금',
    'Payments to Acquire Property, Plant, and Equipment': '유형자산 취득지급',
    'Preferred Stock Dividends and Other Adjustments': '우선주배당 및 기타조정',
    'Preferred Stock, Value, Issued': '발행우선주 자본금',
    'Prepaid Expense and Other Assets, Current': '선급비용 및 기타유동자산',
    'Proceeds from (Payments for) Other Financing Activities': '타재무활동현금유입(유출)',
    'Proceeds from Contributed Capital': '추가납입자본금유입',
    'Proceeds from Contributions from Affiliates': '계열회사납입금유입',
    'Proceeds from Issuance of Common Stock': '보통주발행대금유입',
    'Proceeds from Issuance of Long-term Debt': '장기부채발행대금유입',
    'Proceeds from Maturities, Prepayments and Calls of Debt Securities, Available-for-sale': '만기도래·상환·콜옵션 행사용 매도가능증권대금유입',
    'Proceeds from Noncontrolling Interests': '비지배지분 자금유입',
    'Proceeds from Stock Options Exercised': '스톡옵션 행사대금유입',
    'Property, Plant and Equipment, Net': '유형자산(순)',
    'Property, plant and equipment acquisitions funded by liabilities': '부채로 조달한 유형자산 취득',
    'Provision for Loan, Lease, and Other Losses': '대출·리스 등 손실충당금',
    'Purchases of property and equipment, accrued but unpaid': '미지급 유형자산 취득금액',
    'Real Estate Tax Expense': '부동산 세금비용',
    'Realized Investment Gains (Losses)': '실현투자이익(손실)',
    'Repayments of Long-term Debt': '장기부채 상환액',
    'Restricted Cash and Cash Equivalents': '제한성 현금및현금성자산',
    'Restricted Cash and Investments, Current': '제한성 유동현금 및 투자자산',
    'Restructuring Costs and Asset Impairment Charges': '구조조정비 및 자산손상차손',
    'Retained Earnings (Accumulated Deficit)': '이익잉여금(결손금)',
    'Revenue from Contract with Customer, Including Assessed Tax': '고객계약매출(세금포함)',
    'Share-based Payment Arrangement, Expense': '주식기반보상 비용',
    'Share-based Payment Arrangement, Noncash Expense': '주식기반보상 비현금비용',
    'Short-term Investments': '단기투자자산',
    "Stockholders' Equity Attributable to Noncontrolling Interest": '비지배지분',
    "Stockholders' Equity Attributable to Parent": '지배기업지분',
    'Total Assets': '총자산',
    'Total Liabilities': '총부채',
    'Total interest expense': '총이자비용',
    'Treasury Stock, Common, Value': '자기주식(보통주)',
    'Undistributed Earnings (Loss) Allocated to Participating Securities, Basic': '참여증권귀속미분배이익(손실,기본)',
    'Unrealized Gain (Loss) on Derivatives': '파생상품 미실현이익(손실)',
    'Unrealized Gain (Loss) on Investments': '투자자산 미실현이익(손실)'
}

symbols = load_symbols_from_asset_profile("stock", on_filter=True)

set_identity("honeypipeline@gmail.com")

def get_fundamental(symbol: str, periods=4, period="annual"):
    """재무제표(손익계산서 / 재무상태표 / 현금흐름표)를 하나의 DataFrame으로 반환.

    일부 심볼의 경우 원천 데이터에 'label' 컬럼이 없을 수 있으므로,
    그런 경우 해당 스테이트먼트는 스킵하고 경고만 출력합니다.
    """
    c = Company(symbol)
    dfs = []

    def _prepare(df, statement_type: str):
        if df is None:
            return

        df = df.reset_index()
        df = df.drop(
            columns=["concept", "depth", "is_abstract", "is_total", "section"],
            errors="ignore",
        )
        fy_cols = df.filter(regex=r"^FY").columns
        df = df.dropna(subset=fy_cols, how="all")

        # label 컬럼이 없으면 이 스테이트먼트는 건너뜀
        if "label" not in df.columns:
            print(f"[WARN] {symbol} - {statement_type} 에 'label' 컬럼 없음. 스킵.")
            return

        # 한글 라벨 매핑
        df["label_kr"] = df["label"].map(financial_term_kor_map)
        df["symbol"] = symbol
        df["statement_type"] = statement_type

        # 컬럼 위치 재정렬: symbol → statement_type → ... → label → label_kr → 나머지
        col_symbol = df.pop("symbol")
        df.insert(0, "symbol", col_symbol)
        col_statement_type = df.pop("statement_type")
        df.insert(1, "statement_type", col_statement_type)
        col_label_kr = df.pop("label_kr")
        i = df.columns.get_loc("label") + 1
        df.insert(i, "label_kr", col_label_kr)

        dfs.append(df)

    # 1. Income Statement
    _prepare(c.income_statement(periods=periods, period=period, as_dataframe=True),
             "income_statement")

    # 2. Balance Sheet
    _prepare(c.balance_sheet(periods=periods, period=period, as_dataframe=True),
             "balance_sheet")

    # 3. Cash Flow
    _prepare(c.cashflow_statement(periods=periods, period=period, as_dataframe=True),
             "cashflow")

    # 세 개 다 스킵된 경우 빈 DataFrame 반환
    if not dfs:
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)


not_keys = set()
keys = set(financial_term_kor_map.keys())
for s in symbols[:30]:
    try:
        df = get_fundamental(s)
    except Exception as e:
        print(f"Error processing symbol {s}: {e}")
        continue

    if 'label' not in df.columns:
        print(f"not found column : {s}")
        continue

    new_col = set(c for c in df['label'].to_list() if c not in keys)
    not_keys.update(new_col)  # set에 .update 사용: 중복 없이 추가됨

print(len(not_keys))
display(not_keys)


[WARN] AARD - income_statement 에 'label' 컬럼 없음. 스킵.
[WARN] AARD - balance_sheet 에 'label' 컬럼 없음. 스킵.
[WARN] AARD - cashflow 에 'label' 컬럼 없음. 스킵.
not found column : AARD


No company facts found on url https://data.sec.gov/api/xbrl/companyfacts/CIK0001993344.json
No company facts found on url https://data.sec.gov/api/xbrl/companyfacts/CIK0001993344.json
No company facts found on url https://data.sec.gov/api/xbrl/companyfacts/CIK0001993344.json


not found column : AAUC
[WARN] ABEV - income_statement 에 'label' 컬럼 없음. 스킵.
[WARN] ABEV - cashflow 에 'label' 컬럼 없음. 스킵.
177


{'Accounts Payable, Current',
 'Accounts Receivable',
 'Accounts Receivable, after Allowance for Credit Loss',
 'Accounts Receivable, after Allowance for Credit Loss, Current',
 'Accretion Expense',
 'Accrued Liabilities, Current',
 'Accrued capital expenditures',
 'Accumulated Other Comprehensive (Income) Loss, Defined Benefit Plan, after Tax',
 'Accumulated Other Comprehensive Income (Loss), Net of Tax',
 'Additional Paid in Capital',
 'Additional Paid in Capital, Common Stock',
 'Amortization',
 'Amortization of Debt Issuance Costs',
 'Amortization of Debt Issuance Costs and Discounts',
 'Asset Retirement Obligation, Accretion Expense',
 'Assets',
 'Assets, Current',
 'Bank Owned Life Insurance Income',
 'Cash and Cash Equivalents, at Carrying Value',
 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents',
 'Cash, Cash Equivalents, Restricted Cash and Restricted Cash Equivalents, Period Increase (Decrease), Including Exchange Rate Effect',
 'Cash, Cash Equivalent